conventional method (Convolutional Neural Network and Support Vector Machine)

train CNN

In [3]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, Activation, Flatten, Dense, Input
from keras.datasets import mnist


batch_size = 32
epochs = 15
train_num = 5
test_num = 5000

#load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#
x_train = x_train.reshape((60000,28,28,1))
x_test = x_test.reshape((10000,28,28,1))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = x_train[:train_num,:,:,:]
x_test = x_test[:test_num,:,:,:]
y_train = y_train[:train_num]
y_test = y_test[:test_num]
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model_input = Input(shape=(28, 28, 1))
conv1 = Conv2D(6, (5, 5))(model_input)
acti1 = Activation('linear')(conv1)
conv2 = Conv2D(16, (5, 5))(acti1)
acti2 = Activation('linear')(conv2)

flatten1 = Flatten()(acti2)
dense1 = Dense(2056)(flatten1)
acti5 = Activation('relu')(dense1)
dense2 = Dense(64)(acti5)
acti6 = Activation('relu')(dense2)
dense3 = Dense(10)(acti6)
output = Activation('softmax')(dense3)

model = Model(inputs=model_input, outputs=output)


model.summary()

model.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

history = model.fit(x_train, y_train,  # 画像とラベルデータ
                    batch_size=batch_size,
                    epochs=epochs,     # エポック数の指定
                    verbose=1,         # ログ出力の指定. 0だとログが出ない
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5 train samples
5000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
activation_6 (Activation)    (None, 24, 24, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 16)        2416      
_________________________________________________________________
activation_7 (Activation)    (None, 20, 20, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2056) 

get features from trained CNN

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
from keras.preprocessing import image

train_num_svm=5000
(x_train, y_train), (_, _) = mnist.load_data()

#we use only train data and labels
x_train = x_train.reshape((60000,28,28,1))
x_train = x_train.astype('float32')
x_train /= 255
x_train = x_train[:train_num_svm,:,:,:]
y_train = y_train[:train_num_svm]

get_feature_map = K.function(inputs=[model.input, K.learning_phase()],
                             outputs=[model.layers[3].output])

features, = get_feature_map([x_train, False])
print('features.shape', features.shape)

features.shape (5000, 20, 20, 16)


train SVM with extracted featres

In [8]:
#reshape features for SVM
features = features.transpose(1,2,3,0)
features = features.reshape(-1, features.shape[3])

from sklearn.svm import SVC
clf = SVC(gamma='auto',kernel='linear')
clf.fit(features.T, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
#calculate accuracy of image classification
clf.score(features.T, y_train)

1.0

calculate test accuracy with trained SVM

In [10]:
test_features, = get_feature_map([x_test, False])

#reshape data
test_features = test_features.transpose(1,2,3,0)
test_features = test_features.reshape(-1, test_features.shape[3])

#calculate test accuracy
clf.score(test_features.T, y_test)

0.8764